In [64]:
import pandas as pd
from typing import Dict, List, Union
import pandas as pd
import pandas as pd
from typing import List, Union, Tuple, Dict
from datetime import datetime


In [65]:
def get_dataframe_with_types(path: str, sep: str = ",", columns: Union[List[str], None] = None) -> Tuple[pd.DataFrame, Dict[str, List[str]]]:
    """
    Lê um arquivo CSV e analisa os tipos de dados de suas colunas.
    
    Args:
        path (str): Caminho do arquivo csv.
        sep (str): Separador do arquivo CSV. Defaults to ",".
        columns (Union[List[str], None], optional): Lista opcional das colunas do dataframe. Defaults to None.
        
    Returns:
        Tuple[pd.DataFrame, Dict[str, List[str]]]: Tupla contendo:
            - pd.DataFrame: Dataframe do conteúdo do arquivo csv.
            - Dict[str, List[str]]: Dicionário com tipos de dados e suas respectivas colunas.
    """
    try:
        df = pd.read_csv(path, sep=sep)
        
        if columns is not None:
            df.columns = columns
        
        # Inicializa dicionário de tipos
        column_types = {
            'inteiros': [],
            'decimais': [],
            'texto': [],
            'data': [],
            'timestamp': [],
            'booleanos': [],
            'categoricos': [],
            'outros': []
        }
        
        for column in df.columns:
            # Pega uma amostra não-nula da coluna para análise
            sample = df[column].dropna().head(100)
            if len(sample) == 0:
                column_types['outros'].append(column)
                continue
                
            dtype = df[column].dtype
            
            # Verifica tipos numéricos
            if pd.api.types.is_integer_dtype(dtype):
                column_types['inteiros'].append(column)
            
            elif pd.api.types.is_float_dtype(dtype):
                column_types['decimais'].append(column)
            
            # Verifica tipos de data/hora
            elif pd.api.types.is_datetime64_any_dtype(dtype):
                if df[column].dt.time.iloc[0].strftime('%H:%M:%S') != '00:00:00':
                    column_types['timestamp'].append(column)
                else:
                    column_types['data'].append(column)
            
            elif pd.api.types.is_string_dtype(dtype):
                try:
                    pd.to_datetime(sample, errors='raise')
                    if ':' in str(sample.iloc[0]): 
                        column_types['timestamp'].append(column)
                    else:
                        column_types['data'].append(column)
                except:
                    if df[column].nunique() < 0.2 * len(df):  
                        column_types['categoricos'].append(column)
                    else:
                        column_types['texto'].append(column)
            
            # Verifica booleanos
            elif pd.api.types.is_bool_dtype(dtype):
                column_types['booleanos'].append(column)
                
            else:
                column_types['outros'].append(column)
        
        # Remove chaves com listas vazias
        column_types = {k: v for k, v in column_types.items() if v}
        
        return df, column_types
        
    except Exception as e:
        raise Exception(f"Erro ao processar o arquivo: {str(e)}")

In [66]:
def get_dataframe_metrics(df: pd.DataFrame, column_types: Dict[str, List[str]], include_integers: bool = False) -> List[Dict]:
    """Extrai campos numéricos do DataFrame e calcula suas métricas estatísticas.

    Args:
        df (pd.DataFrame): O dataframe original
        column_types (Dict[str, List[str]]): Dicionário com os tipos de colunas
        include_integers (bool, optional): Se deve incluir campos inteiros. Defaults to False.

    Returns:
        List[Dict]: Lista de dicionários com métricas para cada campo numérico
    """
    # Pega campos decimais
    numeric_fields = column_types.get('decimais', [])
    
    # Adiciona campos inteiros se solicitado
    if include_integers:
        numeric_fields.extend(column_types.get('inteiros', []))
    
    if not numeric_fields:
        return []  # Retorna lista vazia se não houver campos numéricos
    
    # Cria DataFrame apenas com campos numéricos
    df_numeric = pd.DataFrame()
    for field in numeric_fields:
        df_numeric[field] = pd.to_numeric(df[field], errors="coerce")
    
    # Calcula métricas para cada campo
    list_metrics = []
    try:
        for field in df_numeric.columns:
            metrics = {
                field: {
                    "max": float(df_numeric[field].max()),
                    "min": float(df_numeric[field].min()),
                    "avg": float(df_numeric[field].mean()),
                    "sum": float(df_numeric[field].sum()),
                    "median": float(df_numeric[field].median()),
                    "std": float(df_numeric[field].std()),
                    "count": int(df_numeric[field].count()),
                    "null_count": int(df_numeric[field].isna().sum())
                }
            }
            list_metrics.append(metrics)
    except Exception as e:
        raise Exception(f"Erro ao calcular métricas para o campo {field}: {str(e)}")
        
    return list_metrics



In [82]:
import pandas as pd
from typing import Dict, List

def get_dataframe_metrics_date(df: pd.DataFrame, column_types: Dict[str, List[str]]) -> List[Dict]:
    """Extrai campos de data do DataFrame e calcula suas métricas estatísticas.

    Args:
        df (pd.DataFrame): O dataframe original
        column_types (Dict[str, List[str]]): Dicionário com os tipos de colunas

    Returns:
        List[Dict]: Lista de dicionários com métricas para cada campo de data
    """
    # Pega campos de data
    date_fields = column_types.get('data', [])  # Corrigido para 'data'
    
    if not date_fields:
        return []  # Retorna lista vazia se não houver campos de data
    
    # Inicializa a lista para armazenar as métricas
    list_metrics = []
    
    try:
        for field in date_fields:
            df[field] = pd.to_datetime(df[field], errors="coerce")  # Converte para datetime
            
            metrics = {
                field: {
                    "min": df[field].min(),
                    "max": df[field].max(),
                    "count": int(df[field].count()),
                    "null_count": int(df[field].isna().sum())
                }
            }
            list_metrics.append(metrics)
    
    except Exception as e:
        raise Exception(f"Erro ao calcular métricas para os campos de data: {str(e)}")
        
    return list_metrics


In [68]:
def get_dataframe_from_list_dict(list_dict: List[Dict]) -> pd.DataFrame:
    #TODO: melhorar essa função
    return pd.DataFrame(list_dict)


In [83]:
df, tipos = get_dataframe_with_types('dados.csv')

metrics = get_dataframe_metrics(df, tipos)
metrics_date = get_dataframe_metrics_date(df, tipos)

df_metrics = get_dataframe_from_list_dict(metrics)
df_metrics_date = get_dataframe_from_list_dict(metrics_date)

with pd.ExcelWriter('analise_quantitativa_2.xlsx', engine='openpyxl') as writer:
        df_metrics.to_excel(writer, sheet_name='Métricas', index=False)
        df_metrics_date.to_excel(writer, sheet_name='Métricas_date', index=False)

/tmp/ipykernel_273463/4241918616.py:59: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(sample, errors='raise')
